In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import transforms
from PIL import Image
from collections import OrderedDict

%matplotlib inline

Download the pretrained model

In [ ]:
import torchvision.models as models
model = models.googlenet(pretrained = True)

Switch device to gpu if available

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', torch.cuda.get_device_name(torch.cuda.current_device()))

Freeze model's parameters 

In [ ]:
for param in model.parameters():
    param.requires_grad = False 

Upgrading model.

In [ ]:
model.fc = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(1024,512)),
    ('relu', nn.ReLU()),
    ('fc2', nn.Linear(512,2)),
    ('output', nn.LogSoftmax(dim = 1))
    ]))
model #model structure 

I'm using .ImageFolder() as dataloader for train.

In [ ]:
def load_data(data_folder, batch_size, num_workers):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.ColorJitter(),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.Resize(128),
        transforms.ToTensor()
    ])
    
    data = torchvision.datasets.ImageFolder(root = data_folder, transform = transform)
    data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, num_workers = num_workers)
    return data_loader 

The code i used to prepare data for .ImageFolder() on kaggle, started here...

In [ ]:
import zipfile
os.chdir('/kaggle/working/')
file_list = ['/kaggle/input/dogs-vs-cats/train.zip','/kaggle/input/dogs-vs-cats/test1.zip']

for file in file_list:
    zip_ref = zipfile.ZipFile(file, 'r')
    zip_ref.extractall()
    zip_ref.close()

In [ ]:
os.mkdir('/kaggle/working/train/dog')
os.mkdir('/kaggle/working/train/cat')

In [ ]:
import shutil

category = []
filenames = os.listdir('/kaggle/working/train/')
od = '/kaggle/working/train/'
for file in filenames:
    if os.path.isdir(od + file) == False:
        category = file.split('.')[0]
        if category == 'dog':
            shutil.move('/kaggle/working/train/' + file, '/kaggle/working/train/dog/' + file)
        else:
            shutil.move('/kaggle/working/train/' + file, '/kaggle/working/train/cat/' + file)
    else: pass

... and it ended.

Loading train data to the model

In [ ]:
data_folder = '/kaggle/working/train/'
batch_size = 32
num_workers = 0
dataloader = load_data(data_folder, batch_size, num_workers)

In [ ]:
samples, labels = iter(dataloader).next()
plt.figure(figsize=(16,24))
grid_imgs = torchvision.utils.make_grid(samples[:24])
np_grid_imgs = grid_imgs.numpy()

plt.imshow(np.transpose(np_grid_imgs, (1,2,0)))

In [ ]:
model = model.to(device) #shifting model to gpu
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001, amsgrad=True)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[500,1000,1500], gamma=0.5)

Training the model.

In [ ]:
epochs = 3
itr = 1
p_itr = 200
model.train()
total_loss = 0
loss_list = []
acc_list = []
for epoch in range(epochs):
    for samples, labels in dataloader:
        samples, labels = samples.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(samples)
        
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        scheduler.step() 
        
        if itr%p_itr == 0:
            pred = torch.argmax(output, dim=1)
            correct = pred.eq(labels)
            acc = torch.mean(correct.float())
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, acc))
            loss_list.append(total_loss/p_itr)
            acc_list.append(acc)
            total_loss = 0
        
            
        itr += 1

plt.plot(loss_list, label='loss')
plt.plot(acc_list, label='accuracy')
plt.legend()
plt.title('training loss and accuracy')
plt.show()

Loading test data to the model

In [ ]:
test_dir = '/kaggle/working/test1'
test_files = os.listdir(test_dir)

In [ ]:
class CatDogDataset(Dataset):
    def __init__(self, file_list, dir, mode='train', transform = None):
        self.file_list = file_list
        self.dir = dir
        self.mode= mode
        self.transform = transform
        if self.mode == 'train':
            if 'dog' in self.file_list[0]:
                self.label = 1
            else:
                self.label = 0
            
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        img = Image.open(os.path.join(self.dir, self.file_list[idx]))
        if self.transform:
            img = self.transform(img)
        if self.mode == 'train':
            img = img.numpy()
            return img.astype('float32'), self.label
        else:
            img = img.numpy()
            return img.astype('float32'), self.file_list[idx]

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

testset = CatDogDataset(test_files, test_dir, mode='test', transform = test_transform)
testloader = DataLoader(testset, batch_size = 32, shuffle = False, num_workers=0)

In [ ]:
fn_list = []
pred_list = []
for x, fn in testloader:
    with torch.no_grad():
        x = x.to(device)
        output = model(x)
        pred = torch.argmax(output, dim=1)
        fn_list += [n[:-4] for n in fn]
        pred_list += [p.item() for p in pred]

submission = pd.DataFrame({"id":fn_list, "label":pred_list})
submission.head()

Finally testing the model.

In [ ]:
samples, _ = iter(testloader).next()
samples = samples.to(device)
fig = plt.figure(figsize=(24, 16))
fig.tight_layout()
output = model(samples[:24])
pred = torch.argmax(output, dim=1)
pred = [p.item() for p in pred]
ad = {0:'cat', 1:'dog'}
for num, sample in enumerate(samples[:24]):
    plt.subplot(4,6,num+1)
    plt.title(ad[pred[num]])
    plt.axis('off')
    sample = sample.cpu().numpy()
    plt.imshow(np.transpose(sample, (1,2,0)))

So, I training the model on my laptop at GeForce MX110 with 2 Gb VRAM (not allowed) and bath size 24.  
I had this result:   
> [Epoch 1/3] Iteration 200 -> Train Loss: 0.3037, Accuracy: 0.958  
> [Epoch 1/3] Iteration 400 -> Train Loss: 0.2527, Accuracy: 0.958  
> [Epoch 1/3] Iteration 600 -> Train Loss: 0.2506, Accuracy: 0.917  
> [Epoch 1/3] Iteration 800 -> Train Loss: 0.2286, Accuracy: 0.917  
> [Epoch 1/3] Iteration 1000 -> Train Loss: 0.2193, Accuracy: 0.917  
> [Epoch 2/3] Iteration 1200 -> Train Loss: 0.2300, Accuracy: 0.792  
> [Epoch 2/3] Iteration 1400 -> Train Loss: 0.2057, Accuracy: 0.875  
> [Epoch 2/3] Iteration 1600 -> Train Loss: 0.2116, Accuracy: 1.000  
> [Epoch 2/3] Iteration 1800 -> Train Loss: 0.2023, Accuracy: 0.833  
> [Epoch 2/3] Iteration 2000 -> Train Loss: 0.2072, Accuracy: 0.875  
> [Epoch 3/3] Iteration 2200 -> Train Loss: 0.2073, Accuracy: 0.958  
> [Epoch 3/3] Iteration 2400 -> Train Loss: 0.2093, Accuracy: 0.917  
> [Epoch 3/3] Iteration 2600 -> Train Loss: 0.1987, Accuracy: 0.958  
> [Epoch 3/3] Iteration 2800 -> Train Loss: 0.2049, Accuracy: 0.958  
> [Epoch 3/3] Iteration 3000 -> Train Loss: 0.2007, Accuracy: 0.917  

Interestingly, resnet18 shows slightly better results when on the same hardware:
> [Epoch 1/3] Iteration 200 -> Train Loss: 0.2600, Accuracy: 0.875  
> [Epoch 1/3] Iteration 400 -> Train Loss: 0.2020, Accuracy: 0.958  
> [Epoch 1/3] Iteration 600 -> Train Loss: 0.2050, Accuracy: 0.875  
> [Epoch 1/3] Iteration 800 -> Train Loss: 0.1875, Accuracy: 0.875  
> [Epoch 1/3] Iteration 1000 -> Train Loss: 0.1840, Accuracy: 0.833  
> [Epoch 2/3] Iteration 1200 -> Train Loss: 0.1809, Accuracy: 1.000  
> [Epoch 2/3] Iteration 1400 -> Train Loss: 0.1794, Accuracy: 0.958  
> [Epoch 2/3] Iteration 1600 -> Train Loss: 0.1855, Accuracy: 0.958  
> [Epoch 2/3] Iteration 1800 -> Train Loss: 0.1654, Accuracy: 0.833  
> [Epoch 2/3] Iteration 2000 -> Train Loss: 0.1656, Accuracy: 0.833  
> [Epoch 3/3] Iteration 2200 -> Train Loss: 0.1641, Accuracy: 0.958  
> [Epoch 3/3] Iteration 2400 -> Train Loss: 0.1748, Accuracy: 0.917  
> [Epoch 3/3] Iteration 2600 -> Train Loss: 0.1739, Accuracy: 0.958  
> [Epoch 3/3] Iteration 2800 -> Train Loss: 0.1663, Accuracy: 0.958  
> [Epoch 3/3] Iteration 3000 -> Train Loss: 0.1619, Accuracy: 0.958  